In [1]:
import pandas as pd
import numpy as np
import random
import math

import pyltr
from sklearn.model_selection import train_test_split

# Function to sample queries
def sample_queries(df, frac):
    srch_ids = df["srch_id"].unique()[0:int(len(df)*frac)]
    return df.loc[df.srch_id.isin(srch_ids)]

# Train, validation and test set generated from the train set
_train = pd.read_csv("data/processed/_train_undersampled.csv")
_val = pd.read_csv("data/processed/_val.csv")
_test = pd.read_csv("data/processed/_test.csv")

# The actual test set that will be used to submit our final result
test = pd.read_csv("data/processed/kaggle_test.csv")

# The properties that will be used for submitting the final result
Sprops = test["prop_id"]

# To speed up execution we'll just sample for now
_train = sample_queries(_train, 0.1)
_val = sample_queries(_val, 0.2)
_test = sample_queries(_test, 1)

In [2]:
_train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,price_usd_mean,price_usd_median,prop_starrating_mean,prop_starrating_median,prop_review_score_mean,prop_review_score_median
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,50984,2,0.0,...,NaN,0,NaN,0,163.718929,139.05,3.071429,3.0,3.482143,3.5
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,68914,2,3.0,...,11.0,1,114.29,1,163.718929,139.05,3.071429,3.0,3.482143,3.5
2,4,2012-12-31 08:59:22,5,219,NaN,NaN,219,64344,4,3.0,...,NaN,0,NaN,0,265.562500,263.00,3.093750,3.5,3.906250,4.0
3,4,2012-12-31 08:59:22,5,219,NaN,NaN,219,139893,2,3.0,...,NaN,1,NaN,0,265.562500,263.00,3.093750,3.5,3.906250,4.0
4,6,2013-06-05 12:27:51,14,100,NaN,NaN,100,10759,0,2.0,...,NaN,0,NaN,0,125.470000,115.03,1.400000,2.0,3.100000,4.0


In [3]:
_train.shape

(77500, 60)

In [4]:
len(_train.loc[_train.booking_bool == 1]) / len(_train) * 100

31.36258064516129

In [5]:
len(_train.loc[(_train.click_bool == 1) & (_train.booking_bool == 0)]) / len(_train) * 100

19.13290322580645

In [6]:
len(_train.loc[_train.click_bool == 0]) / len(_train) * 100

49.504516129032254

In [7]:
# Generate target label
for df in [_train, _val, _test]:
    df["relevance"] = df.click_bool + df.booking_bool
    df.relevance = df.relevance.map(lambda r: 5 if r > 1 else r)

In [8]:
# Fill missing values (need better methods for each feature)
for df in [_train, _val, _test, test]:
    df.fillna(-1, inplace=True)

In [9]:
# Select properties we care about (kind of arbitrary right now)
features = ["srch_id", "site_id", "prop_country_id", "prop_starrating", "prop_review_score", "prop_brand_bool",
              "prop_location_score1", "prop_location_score2", "price_usd", "promotion_flag", "srch_destination_id",
              "srch_length_of_stay", "srch_adults_count", "srch_children_count", "srch_room_count",
            "prop_starrating_median", "prop_starrating_mean", "price_usd_median", "price_usd_mean",
            "relevance"]

_train = _train[features]
_val = _val[features]
_test = _test[features]
test = test[features[0:-1]]

_train.head()

,srch_id,site_id,prop_country_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_adults_count,srch_children_count,srch_room_count,prop_starrating_median,prop_starrating_mean,price_usd_median,price_usd_mean,relevance
0,1,12,219,2,0.0,0,1.61,-1.0000,85.37,0,23246,1,4,0,1,3.0,3.071429,139.05,163.718929,0
1,1,12,219,2,3.0,1,2.20,0.0206,100.89,0,23246,1,4,0,1,3.0,3.071429,139.05,163.718929,5
2,4,5,219,4,3.0,1,1.79,-1.0000,267.00,0,12588,2,1,0,1,3.5,3.093750,263.00,265.562500,0
3,4,5,219,2,3.0,1,1.61,0.0309,129.00,0,12588,2,1,0,1,3.5,3.093750,263.00,265.562500,1
4,6,14,100,0,2.0,0,1.95,-1.0000,97.63,0,21106,1,2,0,1,2.0,1.400000,115.03,125.470000,0


In [10]:
def split_X_y_qids(df):
    X = df.drop(["srch_id", "relevance"], axis=1)
    y = df.relevance
    qids = df.srch_id
    return (X, y, qids)
    
TX, Ty, Tqids = split_X_y_qids(_train)
VX, Vy, Vqids = split_X_y_qids(_val)
EX, Ey, Eqids = split_X_y_qids(_test)

# Train model
metric = pyltr.metrics.NDCG(k=38)

monitor = pyltr.models.monitors.ValidationMonitor(
    VX, Vy, Vqids, metric=metric, stop_after=100)

model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=100,
    learning_rate=0.1,
    max_features=0.5,
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=64,
    verbose=1
)
model.fit(TX, Ty, Tqids, monitor=monitor)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.7993       0.1776       15.67m      C:      0.3518 B:      0.3518 S:  0
    2       0.8317       0.0317       12.98m      C:      0.3841 B:      0.3841 S:  0
    3       0.8368       0.0040       12.03m      C:      0.3905 B:      0.3905 S:  0
    4       0.8472       0.0100       11.48m      C:      0.4031 B:      0.4031 S:  0
    5       0.8526       0.0042       11.10m      C:      0.4096 B:      0.4096 S:  0
    6       0.8560       0.0022       10.83m      C:      0.4157 B:      0.4157 S:  0
    7       0.8628       0.0077       10.60m      C:      0.4306 B:      0.4306 S:  0
    8       0.8684       0.0057       10.40m      C:      0.4459 B:      0.4459 S:  0
    9       0.8703       0.0018       10.21m      C:      0.4479 B:      0.4479 S:  0
   10       0.8723       0.0005       10.05m      C:      0.4490 B:      0.4490 S:  0
   15       0.8776       0.0015        9.36m      C: 

In [11]:
# Test model locally
Epred = model.predict(EX)

query_groups = pyltr.util.group.get_groups(Eqids)

score = 0.0
i = 0
highest_e = 0
fault_count = 0
for qid, a, b in query_groups:
    sorted_y = pyltr.util.sort.get_sorted_y(Ey[a:b].as_matrix(), Epred[a:b])
    e = metric.evaluate(qid, sorted_y)
    highest_e = max(e, highest_e)
    
    if (e > 1):
        fault_count += 1
    else:
        score += e
        i += 1
        
score /= i
        
score, i, highest_e, fault_count
#metric.calc_mean(Eqids, Ey, Epred)

(0.4912118276400273, 19979, 1.0, 0)

In [12]:
# Predict final submission order
SX = test.drop("srch_id", axis=1)
Sqids = test["srch_id"]
Spred = model.predict(SX)

In [13]:
# Create submission data frame
result = pd.DataFrame()
result["SearchId"] = Sqids
result["PropertyId"] = Sprops
result["Sort"] = Spred
result = result.sort(["SearchId", "Sort"], ascending=False)
result

C:\Users\Richard\AppData\Local\conda\conda\envs\py35\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,SearchId,PropertyId,Sort
6622614,665571,52204,0.983664
6622620,665571,108152,0.810477
6622621,665571,111905,0.564644
6622607,665571,30880,0.530451
6622611,665571,42127,0.317272
6622625,665571,127979,0.308668
6622616,665571,58641,0.143860
6622598,665571,2312,0.123178
6622609,665571,36329,-0.116911
6622600,665571,10681,-0.118344


In [14]:
# Write submission to file
result = result.drop("Sort", axis=1)
result.to_csv("submission.csv", index=False)